In [ ]:
%reload_ext autoreload
%autoreload 2
#import os
#os.environ["MKL_NUM_THREADS"] = "1"
#os.environ["NUMEXPR_NUM_THREADS"] = "1"
#os.environ["OMP_NUM_THREADS"] = "1"

from pathlib import Path
home = str(Path.home())
import sys
import numpy as np
import matplotlib.pyplot as plt
import bct
import sys
sys.path.append(home + '/workspace/networkqit')
import networkqit as nq
from networkqit.graphtheory.models.MEModels import CWTECM, UBCM, UWCM, UECM3
from networkqit.utils.visualization import plot_mle
import pandas as pd
import pickle

In [ ]:
df_thresh_none = pickle.load(open(home+'/workspace/entropy_analysis/notebooks/data_forcellini_thresh_none.pkl','rb'))['forcellini']
G = df_thresh_none[(df_thresh_none['motion']=='L') & (df_thresh_none['passages']==1)]['A'].values[0][0:64,0:64]
G = bct.threshold_proportional(G,0.20)*5
t = G[np.nonzero(G)].min()
print(t)

In [ ]:
W = G
A = (G>t).astype(float)
k = A.sum(axis=0)
s = G.sum(axis=0)
print('threshold=',t)

M = CWTECM(N=len(G), threshold=t)
x0 = (np.concatenate([k,s])) / max(np.max(k),np.max(s))
#x0 = sol['x']
# # Optimize by L-BFGS-B
opt = nq.MLEOptimizer(G, x0=x0, model=M)
import time
start = time.time()
sol = opt.run(model=M, verbose=2, maxiter=5000, ftol=1E-9, gtol=1E-9, method='MLE')
print('Optimization elapsed time = ', time.time()-start, ' seconds ')
print('Loglikelihood = ', M.loglikelihood(G,sol['x']))

pij = M.expected_adjacency(sol['x'])
wij = M.expected_weighted_adjacency(sol['x'])
plot_mle(G,pij,wij)

In [ ]:
plt.figure(figsize=(12,12))
np.random.seed(0)
plt.subplot(2,2,1)
plt.plot((G>0).sum(axis=0), (M.sample_adjacency(sol['x'],batch_size=1000,with_grads=False)>0).mean(0).sum(axis=0),'b.')
plt.plot([0,(G>0).sum(axis=0).max()],[0,(G>0).sum(axis=0).max()],'-k')
plt.grid()
plt.title('with no grads - degrees')

plt.subplot(2,2,2)
plt.plot(G.sum(axis=0), M.sample_adjacency(sol['x'],batch_size=1000,with_grads=False).mean(axis=0).sum(axis=0),'r.')
plt.plot([0,G.sum(axis=0).max()],[0,G.sum(axis=0).max()],'-k')
plt.grid()
plt.yticks(np.arange(0,G.sum(axis=0).max(),5))
plt.title('with no grads - strenght')

plt.subplot(2,2,3)
plt.plot((G>0).sum(axis=0), (M.sample_adjacency(sol['x'],batch_size=1000,with_grads=True)>0).mean(0).sum(axis=0),'b.')
plt.plot([0,(G>0).sum(axis=0).max()],[0,(G>0).sum(axis=0).max()],'-k')
plt.grid()
plt.title('with grads - degrees')

plt.subplot(2,2,4)
plt.plot(G.sum(axis=0), M.sample_adjacency(sol['x'],batch_size=1000,with_grads=True).mean(axis=0).sum(axis=0),'r.')
plt.plot([0,G.sum(axis=0).max()],[0,G.sum(axis=0).max()],'-k')
plt.grid()
plt.title('with grads - strength')

plt.tight_layout()

In [ ]:
df_thresh_none = pickle.load(open(home+'/workspace/entropy_analysis/notebooks/data_forcellini_thresh_none.pkl','rb'))['forcellini']

def optimize_benchmark():
    from itertools import product
    likelihood = []
    #for threshold, motion, passages in product(np.linspace(0.025,0.25,10),['L','M','H'],[ 5, 27,  8, 24,  9,  0,  1, 85,  6]):
    for threshold, motion, passages in product(np.linspace(0.025,0.05,1),['L','M','H'],[9]):
        G = df_thresh_none[(df_thresh_none['motion']==motion) & (df_thresh_none['passages']==passages)]['A'].values[0]
        G = bct.threshold_proportional(G,threshold)
        t = G[np.nonzero(G)].min()
        k = (G>0).sum(axis=0)
        s = (G).sum(axis=0)
        M = CWTECM(N=len(G), threshold=t)
        x0 = (np.concatenate([k,s])) / max(np.max(k),np.max(s))
        # # Optimize by L-BFGS-B
        opt = nq.MLEOptimizer(G, x0=x0, model=M)
        import time
        start = time.time()
        sol = opt.run(model = M, verbose=1, maxiter=100, ftol=1E-9, gtol=1E-6, method='MLE')
        likelihood.append(M.loglikelihood(G,sol['x']))
        print('Loglikelihood = ', M.loglikelihood(G,sol['x']), '\tOptimization elapsed time = ', time.time()-start, ' seconds ')
    return zip(likelihood,product(np.linspace(0.025,0.25,10),['L','M','H'],[ 5, 27,  8, 24,  9,  0,  1, 85,  6]))

In [ ]:
res=list(optimize_benchmark())

In [ ]:
from autograd import grad
h = grad(lambda z : M.sample_adjacency(z,with_grads=True).sum())
gr = h(sol['x'])

In [ ]:
W = np.round(G[0:128,0:128]*50)
A = (G>t).astype(float)
k = A.sum(axis=0)
s = G.sum(axis=0)
print('threshold=',t)

M = UECM3(N=len(G), threshold=t)
x0 = (np.concatenate([k,s])) / max(np.max(k),np.max(s))
#x0 = sol['x']
# # Optimize by L-BFGS-B
opt = nq.MLEOptimizer(G, x0=x0, model=M)
import time
start = time.time()
sol = opt.run(model=M, verbose=2, maxiter=1000, ftol=1E-9, gtol=1E-6, method='MLE')
print('Optimization elapsed time = ', time.time()-start, ' seconds ')
print('Loglikelihood = ', M.loglikelihood(G,sol['x']))

pij = M.expected_adjacency(sol['x'])
wij = M.expected_weighted_adjacency(sol['x'])
plot_mle(G,pij,wij)